In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import cifar100

In [3]:
# dataset cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

169009152/169001437 [==============================] - 2s 0us/step


In [4]:
# data shape
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


In [6]:
# type
type(x_train)

numpy.ndarray

In [7]:
# transfer learning
resnet50 = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(32,32,3),
    pooling='avg'
)

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
resnet50.summary()

In [10]:
dense_layer_1 = tf.keras.layers.Dense(1024, activation='relu')
batch_norm = tf.keras.layers.BatchNormalization()
output = tf.keras.layers.Dense(100)

In [11]:
model = tf.keras.Sequential([
    resnet50, 
    dense_layer_1,
    batch_norm,
    output
])

In [13]:
# epoch, lr
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

In [17]:
linear_cosine = tf.keras.experimental.LinearCosineDecay(0.05, 1000)

In [18]:
cosine_warm_up = tf.keras.experimental.CosineDecayRestarts(0.05, 1000)

In [14]:
learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(schedule = scheduler)

In [15]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

In [21]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(cosine_warm_up),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ["accuracy"]
)

In [22]:
history = model.fit(
    x_train, y_train,
    batch_size = 128,
    epochs=1000,
    callbacks=[early_stopping],
    shuffle=True,
    validation_split=0.2,
)

Epoch 1/1000
313/313 [==============================] - 63s 79ms/step - loss: 6.6980 - accuracy: 0.0104 - val_loss: 6.7197 - val_accuracy: 0.0096
Epoch 2/1000
313/313 [==============================] - 22s 71ms/step - loss: 4.8127 - accuracy: 0.0133 - val_loss: 5.5063 - val_accuracy: 0.0150
Epoch 3/1000
313/313 [==============================] - 22s 70ms/step - loss: 4.3732 - accuracy: 0.0270 - val_loss: 4.3207 - val_accuracy: 0.0286
Epoch 4/1000
313/313 [==============================] - 22s 71ms/step - loss: 4.5980 - accuracy: 0.0293 - val_loss: 30.2669 - val_accuracy: 0.0104
Epoch 5/1000
313/313 [==============================] - 22s 71ms/step - loss: 5.4197 - accuracy: 0.0206 - val_loss: 9.1208 - val_accuracy: 0.0101
Epoch 6/1000
313/313 [==============================] - 22s 71ms/step - loss: 4.7190 - accuracy: 0.0316 - val_loss: 4.5315 - val_accuracy: 0.0368
Epoch 7/1000
313/313 [==============================] - 22s 70ms/step - loss: 4.1449 - accuracy: 0.0598 - val_loss: 11.4857